In [118]:
import pandas as pd
import shutil
import zipfile
import os

from pathlib import Path
from collections import defaultdict

In [119]:
types = {
    0: "Bass",
    1: "Brass",
    2: "Keys",
    3: "Lead",
    4: "Organ",
    5: "Pad",
    6: "Percussion",
    7: "Sequence",
    8: "SFX",
    9: "Strings"
}

In [120]:
types_short = {
    0: "BS",
    1: "BR",
    2: "KS",
    3: "LD",
    4: "OR",
    5: "PD",
    6: "DR",
    7: "SQ",
    8: "FX",
    9: "ST"
}

In [121]:
clean_dir     = Path("../patches/clean")
generated_dir = Path("../patches/generated")
temp_dir      = Path("../patches/temp")

generated_dir.mkdir(parents=True, exist_ok=True)

In [122]:
patches = []

for clean_file in clean_dir.glob("*.csv"):

    patch = pd.read_csv(clean_file)
    patch["bank_name"] = clean_file.stem

    patch = patch[[
        "bank_name",
        "patch_name",
        "patch_type",
        "path"
    ]]

    patch = patch[patch["patch_name"] != "Init"]

    patches.append(patch)

In [123]:
patches = pd.concat(patches)
patches = patches.drop_duplicates(subset=["bank_name", "patch_name", "patch_type"])
patches = patches.reset_index(drop=True)

In [124]:
patches["patch_type_short"]  = patches["patch_type"].map(types_short)
patches["patch_type"]        = patches["patch_type"].map(types)

In [125]:
paths_by_type = defaultdict(list)

for _, row in patches.iterrows():
    patch_type = row["patch_type"]
    patch_path = row["path"]

    src_path = temp_dir / patch_path
    paths_by_type[patch_type].append(src_path)

In [126]:
for patch_type, paths in paths_by_type.items():
    type_dir = generated_dir / patch_type
    subfolder = type_dir / f"01-{patch_type}-A"
    subfolder.mkdir(parents=True, exist_ok=True)

    for i, src_path in enumerate(paths):
        patch_num = i + 1
        new_name = f"01-{patch_type}-A{patch_num}.mbp"
        dst_path = subfolder / new_name
        if src_path.exists():
            shutil.copy(src_path, dst_path)